In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
db_con = sqlite3.connect("../db.sqlite3")
query = """
SELECT datetime, quantity, price, name
FROM restaurant_order
JOIN restaurant_orderitem ON restaurant_order.id = restaurant_orderitem.order_id
JOIN restaurant_product ON restaurant_orderitem.product_id = restaurant_product.id;
"""

data = pd.read_sql(query, db_con)
data.to_csv("../data.csv", index=False)
db_con.close()

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
df = pd.read_csv("../data.csv")
sales = df.groupby("name")["quantity"].sum()
top_10 = sales.sort_values(ascending=False).head(10)
top_10_pers = top_10 / top_10.sum() * 100

plt.figure(figsize = (5, 5))
plt.pie(top_10_pers, labels = top_10_pers.index, autopct='%1.1f%%')
plt.title("Top 10 positions in menu by title")
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df["item_price"] = df["price"] * df["quantity"]
sale_sum = df.groupby("name")["item_price"].sum()
top_10_sum = sale_sum.sort_values(ascending=False).head(10)
top_10_sum_pers = top_10_sum / top_10_sum.sum() * 100

plt.figure(figsize = (5, 5))
plt.pie(top_10_sum_pers, labels=top_10_sum_pers.index, autopct='%1.1f%%')
plt.title("Top 10 positions in menu by price")
plt.show()

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["datetime"] = pd.to_datetime(df["datetime"])
df["order_hour"] = df["datetime"].dt.hour
sales_hour = df.groupby("order_hour")["item_price"].sum()

plt.figure(figsize = (8, 5))
plt.bar(sales_hour.index, sales_hour.values)
plt.title("Sales by hour")
plt.xlabel("Hour")
plt.ylabel("Summ")
plt.show()

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df["week_day"] = df["datetime"].dt.dayofweek
day_sales = df.groupby("week_day")["item_price"].sum()
day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_sales.index = day_names

plt.figure(figsize = (8, 5))
plt.bar(day_sales.index, day_sales.values)
plt.title("Sales by day")
plt.xlabel("Day")
plt.ylabel("Summ")
plt.show()